# Testing
In this notebook, we will test the model of choice.

In [1]:
### Michael Engel ### 2022-04-25 ### main.ipynb ###
### adapted by Niklas Eisl, Colin Moldenhauer, 2022/23 ###
%matplotlib inline

import os
import platform
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import torch
from tensorboardX import SummaryWriter

from eolearn.core import FeatureType

from libs.ConfigME import Config, importME, get_most_recent_config
from libs.QuantileScaler_eolearn import QuantileScaler_eolearn_tdigest
from libs.Dataset_eolearn import Dataset_eolearn
from libs import AugmentME

print("Working Directory:",os.getcwd())

# Config
First, we load our configuration file which provides all information we need throughout the script.

In [2]:
config_name = get_most_recent_config(".", pattern="config_.*[.]dill", mode="m")
config = Config.LOAD(config_name)
print("loaded config:", config_name)

# Batch Size
As we deal with high dimensional data and several time stamps, it is important to choose a reasonably low batch size. A too small batch size on the other hand will result in highly stochastic gradients. We have found a batch size of 6 to work well for this task.

In [3]:
print(f'Chosen batch_size: {config["batch_size"]}')

Chosen batch_size: 6


# Quantile Scaling
As discussed in the second notebook, we want to apply quantile scaling to our data.
We load the scaler that we have already defined.

In [4]:
Scaler = QuantileScaler_eolearn_tdigest.LOAD(os.path.join(config["dir_results"],config["savename_scaler"]))

# Dataloader
First, we need to get the paths for all samples within our training and validation datasets, respectively. More specifically, we create a list of paths that will then be passed to the `Dataset_eolearn` class which prepares the data in a way such that the dataloader can load it into the batches that we use during training and validation.

In [5]:
paths_test = [os.path.join(config["dir_test"], file).replace("\\","/") for file in os.listdir(config["dir_test"])]

Now, we are ready to define our datasets using the `Dataset_eolearn` class!

Remember that PyTorch asks for the shape `[batch_size x channels x timestamps x height x width]`.
The `Quantile
_eolearn_tdigest` handles this by setting `transform=Torchify(1)`.
For the reference and the mask, we use the `Torchify` class as provided from the `Dataset_eolearn` package, too.

So befor we initialize the Datasets, let's briefly have a look at the `Torchify` class. In a nutshell, this class handles our final transforms such that Pytorch can use the data for training. This includes rearranging of dimensions and removing of `NaN` values.

In [6]:
def torchify(array):
    return np.moveaxis(array, -1, 0)

def nan_to_value(array, value=0):
    nan_loc = np.isnan(array)
    array[nan_loc] = value
    return array


class Torchify():
    def __init__(self,squeeze=False, nanvalue=0):
        self.squeeze = squeeze
        self.nanvalue = nanvalue
    def __call__(self,array):
        array_ = torchify(array)
        array_ = nan_to_value(array_, value=self.nanvalue)

        if self.squeeze==True:
            return array_.squeeze()
        elif type(self.squeeze)==int:
            return array_.squeeze(self.squeeze)
        else:
            return array_

In [7]:
dataset_test = Dataset_eolearn(
    paths = paths_test,
    feature_data = (FeatureType.DATA, "data"),
    feature_reference = (FeatureType.DATA, "reference"),
    feature_mask = (FeatureType.MASK, "mask_reference"),

    transform_data = Scaler,
    transform_reference = Torchify(squeeze=1, nanvalue=0),
    transform_mask = Torchify(squeeze=1, nanvalue=0),
    
    return_idx = False,
    return_path = False,

    torchdevice = None,
    torchtype_data = torch.FloatTensor,
    torchtype_reference = torch.FloatTensor,
    torchtype_mask = torch.FloatTensor,
)

Let's test our datasets! 

As we can see, each batch of our input data `x` consists of 
- 6 samples, 
- 6 channels (where the channels correpond to the downloaded bands `B02`, `B03`, `B04`, `B08`, `B8A` and `B11`),
- 8 consecutive time stamps
- and a spatial resolution of `256x256` for each patch.

The corresponding reference `y` collapses the channel and timestamp dimesions and we end up with
- 6 samples
- and a spatial resolution of `256x256` for each patch, where all values are in the range `[-1,1]` to represent the NDVI index.

The mask has the same shape as the reference `y` with binary values to filter with the`no-data mask` and `cloud mask`.

In [8]:
sample_train = dataset_test[:config["batch_size"]]
print('Test Data Shape:', sample_train[0].shape)
print('Test Reference Shape:', sample_train[1].shape)
print('test Mask Shape:', sample_train[2].shape)

Test Data Shape: torch.Size([6, 6, 8, 256, 256])
Test Reference Shape: torch.Size([6, 256, 256])
test Mask Shape: torch.Size([6, 256, 256])


Let's define our dataloader for each dataset.
We will double our `batch_size` for testing as no gradient calculation is needed here.

In [9]:
dataloader_test = torch.utils.data.DataLoader(
    dataset = dataset_test,
    batch_size = config["batch_size"],
    shuffle = True,
    sampler = None,
    batch_sampler = None,
    num_workers = 0 if platform.system()=="Windows" else config["threads"],
    collate_fn = None,
    pin_memory = False,
    drop_last = True,
    timeout = 0,
    worker_init_fn = None,
    multiprocessing_context = None
)

# Model
Now, it's time to initialise our model.
We will do that using `importME` since we want to keep flexibility with regard to the model architecture used. That way, changes can easily be made in the Configuaration Notebook without having to modify this Notebook.

In [10]:
module_model = importME(config["module_model"])

In [11]:
model = module_model(**config["kwargs_model"])
model

ConvLSTM(
  (cell_list): ModuleList(
    (0): ConvLSTMCell(
      (conv): Conv2d(26, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvLSTMCell(
      (conv): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ConvLSTMCell(
      (conv): Conv2d(21, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

# Training
Before we can start training our model, we have to define a loss function.
We will keep it as flexible as the model itself and use `importME`.

As we deal with a regression task we use a classical `MSE Loss` function. However, it is important to also take the mask into account that we get for each batch in addition to `x` and `y`. The loss fuction can be modified such that no loss is computed for those pixels where the mask has a value of 1.

We have implemented such a custom loss called `MSELossMasked` which can be found in `libs\loss.py`.

In [12]:
loss_function = importME(config["module_loss"])(**config["kwargs_loss"])

Incorporating utils!


No optimization without an optimizer! 
Due to corresponding device issues, we have to send our model to the device before we define our optimizer.

In [13]:
model.to(config["device"])

ConvLSTM(
  (cell_list): ModuleList(
    (0): ConvLSTMCell(
      (conv): Conv2d(26, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvLSTMCell(
      (conv): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ConvLSTMCell(
      (conv): Conv2d(21, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

Now, we can define our optimizer with the parameters already been sent to our chosen device!

# Testing
To assess the performance of our model, we load some metrics. Similar to the loss function, we use the `Masked MSE Loss` function as our first metric and a `Squared Variance Error` to also take the variance into account. The implementatin of both metrics can be found in `libs\metrics.py`

In [14]:
metric = importME(config["module_metric"])

Of course, we would like to track the proceeding of our testing procedure.
Hence, we define a tensorboard [SummaryWriter](https://tensorboardx.readthedocs.io/en/latest/tensorboard.html#tensorboardX.SummaryWriter).

In [15]:
writer = SummaryWriter(config["dir_tensorboard"])

Anyway, we would like to make our experiment reproducible.
Thus, we set the seeds such that all random number generation and shuffling is done in a deterministic manner.

In [16]:
np.random.seed(config["seed"])
torch.manual_seed(config["seed"])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

After the long training, we would like to test our model on the chosen test tiles.

However, we load the model providing the best validation loss.
For that, we will use the `BaseClass` from AugmentME.

In [17]:
model = AugmentME.BaseClass(mode="torch")
model.load(os.path.join(config["dir_results"],config["model_savename_bestloss"]))

loadME_torch: start loading of the entire model!
loadME_torch: loaded


True

Since the testing script is rather similar to the validation part of our training procedure, we do not discuss this here.

In [ ]:
print('Start testing...')
model.eval()
losss_test = []
accs_test = []
weights_test = []
with torch.no_grad():
    for step_test, (x_test, y_test, mask_test) in enumerate(dataloader_test):
        #%%%% clean cache of GPU
        torch.cuda.empty_cache()

        #%%%% forward pass
        out, _ = model(x_test)
        pred_test = out[0][:, -1, ...].squeeze()

        #%%%% compute loss
        loss_test = loss_function(pred_test, y_test, mask_test)

        if type(metric)==list:
            test_acc = [metric_(pred_test, y_test, mask_test).cpu().detach() for metric_ in metric]
        else:
            test_acc = metric(pred_test, y_test, mask_test).cpu().detach()

        #%%%% printing stuff
        print(
            "[{}] Test Step: {:d}/{:d}, \tbatch_size: {} \tLoss: {:.4f} \tAcc: {}".format(
                dt.datetime.now().strftime("%Y-%m-%dT%H-%M-%S"),
                step_test+1,
                len(dataloader_test),
                dataloader_test.batch_size,
                loss_test.mean(),
                {metric_.__name__:test_acc_ for metric_,test_acc_ in zip(metric,test_acc)} if type(metric)==list else test_acc
            )
        )

        #%%%% collect loss and accuracy
        losss_test.append(loss_test.cpu().detach().numpy())
        accs_test.append(test_acc)
        weights_test.append(torch.count_nonzero(mask_test).cpu().detach().numpy())

        #%% plot
        fig, axis = plt.subplots(nrows=2, ncols=dataloader_test.batch_size,
                                     figsize=(3*dataloader_test.batch_size,2*3))

        axis[0][0].set_ylabel("Prediction")
        axis[1][0].set_ylabel("Reference")
        for i in range(dataloader_test.batch_size):

            axis[0][i].imshow(pred_test[i].squeeze().cpu().detach(), vmin=-1,vmax=1,cmap="Greens")
            axis[0][i].set_xticks([])
            axis[0][i].set_yticks([])

            axis[1][i].imshow(y_test[i].squeeze().cpu().detach().numpy(), vmin=-1,vmax=1,cmap="Greens")
            axis[1][i].set_xticks([])
            axis[1][i].set_yticks([])

        plt.tight_layout()
        plt.show()
        plt.savefig(fname=os.path.join(config["dir_imgs_test"], "Test%i" % step_test), dpi="figure")
        writer.add_figure(tag="PredictionTest", figure=fig, global_step=step_test, close=True, walltime=None)

    #%%%% total loss and accuracy
    total = np.sum([np.sum(weight_) for weight_ in weights_test])
    loss_test_total = np.sum([weight_/total*loss_ for weight_,loss_ in zip(weights_test,losss_test)])
    if type(metric)==list:
        acc_test_total = [np.sum([weight_/total*acc_[i] for weight_,acc_ in zip(weights_test,accs_test)]) for i in range(len(metric))]
    else:
        acc_test_total = np.sum([weight_/total*acc_ for weight_,acc_ in zip(weights_test,accs_test)])

    # print total values
    print(
        "[{}] Test: \tTotal Loss: {:.4f} \tTotal Acc: {}".format(
            dt.datetime.now().strftime("%Y-%m-%dT%H-%M-%S"),
            loss_test_total,
            {metric_.__name__:test_acc_ for metric_,test_acc_ in zip(metric,acc_test_total)} if type(metric)==list else acc_test_total
        )
    )

    #%%% write to tensorboard
    #%%%% log loss
    writer.add_scalar(f'LossTest/{type(loss_function).__name__}', loss_test_total, global_step=step_test)

    #%%%% log metric
    if type(metric)==list:
        writer.add_scalars('AccuracyTest',{metric_.__name__:test_acc_ for metric_,test_acc_ in zip(metric,acc_test_total)}, global_step=step_test)
    else:
        writer.add_scalar('AccuracyTest', acc_test_total, global_step=step_test)